In [74]:
import pandas as pd

In [75]:
# Read the data 
rows_to_skip = [0, 1, 2, 3, 5]
df = pd.read_csv('2023-07-01-2023-07-31_Railroad Monthly.csv',
                 skiprows=rows_to_skip, header=0)

df

,Timestamp,POA Sensor,Weather station module temperature,Weather station ambient temperature,Max wind speed - Weather Station w/ Module & POA LP02,Production meter L-N voltage,Production meter L-L voltage,Production meter active power,"Schneider INV-1A, Line kW","Schneider INV-1B, Line kW","Schneider INV-2A, Line kW","Schneider INV-2B, Line kW","Schneider INV-3A, Line kW","Schneider INV-3B, Line kW"
0,7/1/2023 12:00:00 AM,0.0,75.88164,77.90759,0.0,13635.00,23617.67,-5.248717,0.0,0.0,0.0,0.0,0.0,0.0
1,7/1/2023 12:15:00 AM,0.0,74.96099,77.38332,0.0,13619.67,23589.33,-5.233324,0.0,0.0,0.0,0.0,0.0,0.0
2,7/1/2023 12:30:00 AM,0.0,75.38254,77.32322,0.0,13577.33,23515.67,-5.204318,0.0,0.0,0.0,0.0,0.0,0.0
3,7/1/2023 12:45:00 AM,0.0,75.10126,77.07894,0.0,13572.00,23506.67,-5.201568,0.0,0.0,0.0,0.0,0.0,0.0
4,7/1/2023 1:00:00 AM,0.0,74.25250,76.61016,0.0,13585.67,23530.00,-5.208205,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,7/31/2023 10:45:00 PM,0.0,65.83338,68.41074,0.0,13626.00,23600.33,-5.054621,0.0,0.0,0.0,0.0,0.0,NaN
2972,7/31/2023 11:00:00 PM,0.0,65.38278,68.05986,0.0,13634.33,23613.67,-5.062860,0.0,0.0,0.0,0.0,0.0,NaN
2973,7/31/2023 11:15:00 PM,0.0,64.97095,67.79126,0.0,13643.33,23632.00,-5.065029,0.0,0.0,0.0,0.0,0.0,NaN
2974,7/31/2023 11:30:00 PM,0.0,64.98528,67.59441,0.0,13626.67,23602.00,-5.057930,0.0,0.0,0.0,0.0,0.0,NaN


In [76]:
if df['POA Sensor'].isna().any():
    df['POA Sensor'] = df['POA Sensor'].fillna(-999)
    print("The Irradiance column has missing values. Please document the issue in the Data Issues spreadsheet.")
else:
    print("No missing value detected in the Irradiance column ")

df

The Irradiance column has missing values. Please document the issue in the Data Issues spreadsheet.


,Timestamp,POA Sensor,Weather station module temperature,Weather station ambient temperature,Max wind speed - Weather Station w/ Module & POA LP02,Production meter L-N voltage,Production meter L-L voltage,Production meter active power,"Schneider INV-1A, Line kW","Schneider INV-1B, Line kW","Schneider INV-2A, Line kW","Schneider INV-2B, Line kW","Schneider INV-3A, Line kW","Schneider INV-3B, Line kW"
0,7/1/2023 12:00:00 AM,0.0,75.88164,77.90759,0.0,13635.00,23617.67,-5.248717,0.0,0.0,0.0,0.0,0.0,0.0
1,7/1/2023 12:15:00 AM,0.0,74.96099,77.38332,0.0,13619.67,23589.33,-5.233324,0.0,0.0,0.0,0.0,0.0,0.0
2,7/1/2023 12:30:00 AM,0.0,75.38254,77.32322,0.0,13577.33,23515.67,-5.204318,0.0,0.0,0.0,0.0,0.0,0.0
3,7/1/2023 12:45:00 AM,0.0,75.10126,77.07894,0.0,13572.00,23506.67,-5.201568,0.0,0.0,0.0,0.0,0.0,0.0
4,7/1/2023 1:00:00 AM,0.0,74.25250,76.61016,0.0,13585.67,23530.00,-5.208205,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,7/31/2023 10:45:00 PM,0.0,65.83338,68.41074,0.0,13626.00,23600.33,-5.054621,0.0,0.0,0.0,0.0,0.0,NaN
2972,7/31/2023 11:00:00 PM,0.0,65.38278,68.05986,0.0,13634.33,23613.67,-5.062860,0.0,0.0,0.0,0.0,0.0,NaN
2973,7/31/2023 11:15:00 PM,0.0,64.97095,67.79126,0.0,13643.33,23632.00,-5.065029,0.0,0.0,0.0,0.0,0.0,NaN
2974,7/31/2023 11:30:00 PM,0.0,64.98528,67.59441,0.0,13626.67,23602.00,-5.057930,0.0,0.0,0.0,0.0,0.0,NaN


In [77]:
if df['Production meter active power'].isna().any():
    # Specific situations should be considered here.
    print("The Production meter active power column has missing values. Please document the issue in the Data Issues spreadsheet.")
else:
    print("No missing value detected in the Production meter active power column ")

No missing value detected in the Production meter active power column 


In [78]:
def check_missing_values(column_name, threshold_col='POA Sensor', threshold_val=100):
    condition = (df[threshold_col] > threshold_val) & (df[column_name].isna())
    if condition.any():
        print(f"There are missing values in the {column_name} column when {threshold_col} is above {threshold_val} W/m2.")
        missing_indices = df[condition].index
        for index in missing_indices:
            print(f"Missing value at Index: {index}")
        df.loc[condition, column_name] = -999
    else:
        print(f"There are no missing values in the {column_name} column when {threshold_col} is above {threshold_val} W/m2.")

check_missing_values('Weather station ambient temperature')
check_missing_values('Max wind speed - Weather Station w/ Module & POA LP02')


There are no missing values in the Weather station ambient temperature column when POA Sensor is above 100 W/m2.
There are missing values in the Max wind speed - Weather Station w/ Module & POA LP02 column when POA Sensor is above 100 W/m2.
Missing value at Index: 1187
Missing value at Index: 1188
Missing value at Index: 1189
Missing value at Index: 1190
Missing value at Index: 1191
Missing value at Index: 1192
Missing value at Index: 1193
Missing value at Index: 1194
Missing value at Index: 1195
Missing value at Index: 1196
Missing value at Index: 1202
Missing value at Index: 1203


In [79]:
df.to_csv('investigate.csv', index=False)